In [1]:
import sapien.core as sapien
from sapien.core import Pose
from Tools import misc, ForwardKinematics, ModelSim, NumForwardDynamicsDer, MathForwardDynamics
import numpy as onp
import jax.numpy as np
from jax import jit, jacfwd, jacrev, grad
from ilq import ILQG
import timeit
import os
import matplotlib.pyplot as plt

os.environ["XLA_PYTHON_CLIENT_MEM_FRACTION"] = "0.4"
os.environ["XLA_FLAGS"] = ("--xla_cpu_multi_thread_eigen=false "
                           "intra_op_parallelism_threads=12")

sim = sapien.Engine()
renderer = sapien.OptifuserRenderer()
sim.set_renderer(renderer)
render_controller = sapien.OptifuserController(renderer)

Using default glsl path /home/zack/anaconda3/envs/ml/lib/python3.7/site-packages/sapien/glsl_shader/130


In [2]:
DEBUG = False


def create_scene(timestep, visual):
    s = sim.create_scene()
    s.add_ground(-1)
    s.set_timestep(timestep)

    loader = s.create_urdf_loader()
    loader.fix_root_link = False
    if visual:
        loader.collision_is_visual = True
        s.set_ambient_light([0.5, 0.5, 0.5])
        s.set_shadow_light([0, 1, -1], [0.5, 0.5, 0.5])

    # build
    robot = loader.load("../assets/Humanoid/humanoid.urdf")

    robot.set_root_pose(Pose([0,0,-0.7]))

    for joint in robot.get_joints():
        joint.set_drive_property(stiffness=0, damping=10)

    return s, robot


# renderer_timestep = 1 / 60
optim_timestep = 1 / 500
sim_timestep = 1 / 60
render_steps = int(sim_timestep / optim_timestep) + 1
s0, robot = create_scene(optim_timestep, True)

render_controller.set_camera_position(-5, 0, 0)
render_controller.set_current_scene(s0)
render_controller.show_window()

In [3]:
factor = np.array([200] * robot.dof)
u_range = np.array([-factor, factor])
render_controller.focus(robot.get_links()[0])
while not render_controller.should_quit:
    u = onp.random.random(robot.dof) * factor * 2 - factor
    u = np.clip(u, u_range[0], u_range[1])
    robot.set_qf(u)
    for _ in range(render_steps):
        s0.step()
    s0.update_render()
    render_controller.render()

TypeError: set_qf(): incompatible function arguments. The following argument types are supported:
    1. (self: sapien.core.pysapien.ArticulationBase, qf: numpy.ndarray[float32]) -> None

Invoked with: <sapien.core.pysapien.Articulation object at 0x7f8718021630>, DeviceArray([-123.38324 ,  142.83313 ,  -63.891647,   47.163223,
              -30.839325,   32.430603,  179.74063 ,  -51.49945 ,
              193.77869 ,  103.16217 ,  -77.5741  , -136.06192 ,
              140.39963 ,  -19.282684,   48.98123 ,   22.568207,
             -166.49881 , -124.77914 , -108.85042 ,   77.34802 ,
              -55.14804 ], dtype=float32)

In [ ]:
robot.get_root_pose()


